In [4]:
%load_ext autoreload
%autoreload 2

In [1]:
from math import sin
from functools import reduce

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.nn import ModuleList as mdl
import torch.optim as optim
import torch.nn.functional as F
import tqdm

In [3]:
# functions to test
def parity(x):
    return x % 2


def compose(fs):
    def compose2(f, g):
        return lambda *a, **kw: f(g(*a, **kw))
    return reduce(compose2, fs)


# makes two-column dataset, first is data input to function of choice, second gets replaced w/ function output
def make_xs(n):    
    return np.random.randint(0, 10, (n, 2))


# calls function of choice, f
def make_data(n, f, dtype=None):
    xs = make_xs(n) if dtype is None else make_xs(n).astype(dtype)
    xs[:, 1] = f(xs[:, 0])
    xs, ys = xs[:, 0], xs[:, 1]
    return xs, ys


# TODO: add batches
def make_data_parity(n):
    xs_train, ys_train = make_data(n, parity)
    xs_test, ys_test = make_data(n // 10, parity)
    return xs_train, ys_train, xs_test, ys_test


def make_data_sin(n):
    xs_train, ys_train = make_data(n, np.sin, dtype=np.float32)
    xs_test, ys_test = make_data(n // 10, np.sin, dtype=np.float32)
    return xs_train, ys_train, xs_test, ys_test


def make_loader(xs, ys, batch_size):
    xs, ys = torch.tensor(xs, dtype=torch.float32, requires_grad=True), torch.tensor(ys, dtype=torch.float32, requires_grad=True)
    data = list(zip(xs, ys))
    loader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=True)
    return loader


def euclidean_distance(x, y):
    return torch.sqrt(torch.sum((x - y) ** 2))


def averager(f):
    return lambda x, y: f(x, y) / len(x)


def accuracy(x, y):
    acc = torch.sum(x == y) / len(x)
    return acc.item()


n = 1000

xs_train_parity, ys_train_parity, xs_test_parity, ys_test_parity = make_data_parity(n)
xs_train_sin, ys_train_sin, xs_test_sin, ys_test_sin = make_data_sin(n)


print("parity:", list(zip(xs_train_parity[:5], ys_train_parity[:5])))

print("sin:", list(zip(xs_train_sin[:5], ys_train_sin[:5])))

parity: [(np.int64(2), np.int64(0)), (np.int64(1), np.int64(1)), (np.int64(7), np.int64(1)), (np.int64(1), np.int64(1)), (np.int64(6), np.int64(0))]
sin: [(np.float32(7.0), np.float32(0.6569866)), (np.float32(7.0), np.float32(0.6569866)), (np.float32(5.0), np.float32(-0.9589243)), (np.float32(4.0), np.float32(-0.7568025)), (np.float32(3.0), np.float32(0.14112))]


In [73]:
torch.mean(torch.sqrt(torch.sum(torch.tensor([1, 2, 3]) ** 2)))

tensor(3.7417)

In [74]:
averager(euclidean_distance)(torch.tensor([1, 3]), torch.tensor([3, 7]))

tensor(2.2361)

In [2]:
from wxml.mlp import MLP


def train_step(model, loss_fn, opt, x, y):
    assert model.training and x.requires_grad and y.requires_grad
    opt.zero_grad()
    y_pred = model(x)
    loss = loss_fn(y_pred, y)
    loss.backward()
    opt.step()
    return y_pred, loss.item()
    

def train(model, loss_fn, opt, loader, epochs=10):
    for epoch in (pbar := tqdm.trange(epochs, desc="loss: inf, acc: inf")):
        for x, y in loader:
            y_pred, loss = train_step(model, loss_fn, opt, x, y)
            acy = accuracy(y_pred, y)
            pbar.set_description(f"loss: {loss:.3f}, acy: {acy:.3f}")
    
lr = 1e-3

num_layers = 2
input_dim = 1
output_dim = 1
hidden_dim = 10

model = MLP(num_layers, input_dim, hidden_dim, output_dim)
modek = compose([round, model])
opt = optim.SGD(model.parameters(), lr=lr)
loader_parity = make_loader(xs_train_parity, ys_train_parity, 32)

NameError: name 'compose' is not defined

In [76]:
epochs = 100

train(model, averager(euclidean_distance), opt, loader_parity, epochs=epochs)

loss: 0.511, acc: 0.000: 100%|██████████| 100/100 [00:06<00:00, 14.40it/s]


In [78]:
def round(x):
    if x >= 0.5: x = 1
    else: x = 0
    return x

def evaluate(model, loss_fn, loader):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    total_acc = 0
    count = 0
    
    with torch.no_grad():  # Disable gradient computation
        for x, y in loader:
            # print("test:", model(x))
            # y_pred = [round(tt) for tt in model(x)] # round to the nearest integer 
            #print(round(model(x)[0][0]))
            y_pred = torch.tensor([round(tt) for tt in model(x)])
            print(y_pred)
            loss = loss_fn(y_pred, y)
            acc = accuracy(y_pred, y)
            
            total_loss += loss.item() * len(x)
            total_acc += acc * len(x)
            count += len(x)
    
    avg_loss = total_loss / count
    avg_acc = total_acc / count
    
    print(f"Test Loss: {avg_loss:.3f}, Test Accuracy: {avg_acc:.3f}")
    return avg_loss, avg_acc

print(xs_test_parity[:5], ys_test_parity[:5])
# Convert test data into DataLoader
loader_test_parity = make_loader(xs_test_parity, ys_test_parity, batch_size=32)

# Evaluate the model
evaluate(model, averager(euclidean_distance), loader_test_parity)


[2 5 9 4 9] [0 1 1 0 1]
tensor([1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1,
        0, 1, 0, 1, 1, 0, 0, 0])
tensor([0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 1, 0, 0, 1, 0, 0])
tensor([0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0])
tensor([0, 0, 0, 1])
Test Loss: 0.139, Test Accuracy: 0.430


(0.1386343002319336, 0.43)